In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import pandas as pd
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join
from threading import Thread

class MultiFactorAuthApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Многофакторная Аутентификация")

        self.register_button = tk.Button(root, text="Регистрация", command=self.open_registration_window)
        self.register_button.pack(pady=10)

        self.login_button = tk.Button(root, text="Вход", command=self.open_login_window)
        self.login_button.pack(pady=10)

        self.data_file = "users.csv"
        if not os.path.exists(self.data_file):
            self.create_user_file()

        # Проверка наличия файла каскада
        self.face_classifier_path = 'haarcascade_frontalface_default.xml'
        if not os.path.isfile(self.face_classifier_path):
            messagebox.showerror("Error", "Файл haarcascade_frontalface_default.xml не найден!")
            self.root.destroy()
            return
        
        # Инициализация face_classifier
        self.face_classifier = cv2.CascadeClassifier(self.face_classifier_path)


    def create_user_file(self):
        users = pd.DataFrame(columns=["ID", "login", "password"])
        users.to_csv(self.data_file, index=False)

    def open_registration_window(self):
        self.registration_window = tk.Toplevel(self.root)
        self.registration_window.title("Регистрация")
    
        tk.Label(self.registration_window, text="Логин:").pack()
        self.login_entry = tk.Entry(self.registration_window)
        self.login_entry.pack()
    
        tk.Label(self.registration_window, text="Пароль:").pack()
        self.password_entry = tk.Entry(self.registration_window, show='*')
        self.password_entry.pack()
    
        # Текст о сборе биометрических данных
        tk.Label(self.registration_window, text="После регистрации будет осуществлён сбор биометрических персональных данных.").pack(pady=10)
    
        # Флажок согласия
        self.consent_var = tk.BooleanVar()
        consent_checkbox = tk.Checkbutton(self.registration_window, text="Я ознакомлен с согласием на обработку биометрических персональных данных", variable=self.consent_var)
        consent_checkbox.pack()
    
        # Кнопка для открытия текста согласия
        consent_button = tk.Button(self.registration_window, text="Согласие", command=self.show_consent)
        consent_button.pack(pady=5)
    
        self.register_confirm_button = tk.Button(self.registration_window, text="Подтвердить", command=self.register_user)
        self.register_confirm_button.pack(pady=10)
    
    def show_consent(self):
        consent_window = tk.Toplevel(self.root)
        consent_window.title("Согласие на обработку биометрических персональных данных")
    
        try:
            with open('bio.txt', 'r', encoding='utf-8') as file:
                consent_text = file.read()
                tk.Label(consent_window, text=consent_text, wraplength=400, justify="left").pack(padx=10, pady=10)
        except FileNotFoundError:
            messagebox.showerror("Error", "Файл bio.txt не найден.")
            consent_window.destroy()
    
        tk.Button(consent_window, text="Закрыть", command=consent_window.destroy).pack(pady=5)


    def register_user(self):
        # Проверка на согласие
        if not self.consent_var.get():
            messagebox.showwarning("Warning", "Пожалуйста, ознакомьтесь и согласитесь с обработкой биометрических персональных данных.")
            return
    
        login = self.login_entry.get().strip()  # Удаляем пробелы в начале и конце
        password = self.password_entry.get().strip()  # Удаляем пробелы в начале и конце
    
        if not login or not password:
            messagebox.showwarning("Warning", "Пожалуйста, заполните все поля!")
            return
    
        users = pd.read_csv(self.data_file)
        if login in users['login'].values:
            messagebox.showwarning("Warning", "Этот логин уже существует!")
            return
    
        user_id = len(users) + 1
        new_user = pd.DataFrame({"ID": [user_id], "login": [login], "password": [str(password)]})
        users = pd.concat([users, new_user], ignore_index=True)
        users.to_csv(self.data_file, index=False)
    
        os.makedirs('./photo', exist_ok=True)
        self.capture_photos(user_id)
        self.train_model()
    
        messagebox.showinfo("Info", "Регистрация завершена!")
        self.registration_window.destroy()



    def capture_photos(self, user_id):
        cap = cv2.VideoCapture(0)
        count = 0

        while count < 1000:
            ret, frame = cap.read()
            if ret:
                cv2.imshow("Capturing Photos", frame)
                face = self.face_extractor(frame)
                if face is not None:
                    count += 1
                    file_name_path = f'./photo/{user_id}.{count}.jpg'
                    cv2.imwrite(file_name_path, face)
                if cv2.waitKey(1) == 27:  # ESC для выхода
                    break

        cap.release()
        cv2.destroyAllWindows()

    def face_extractor(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    def train_model(self):
        data_path = './photo/'
        onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    
        Training_Data, Labels = [], []
        for i, files in enumerate(onlyfiles):
            image_path = join(data_path, onlyfiles[i])
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            label = onlyfiles[i].split(".")[0]
    
            # Изменяем размер изображения
            images = cv2.resize(images, (200, 200))
            
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(label)
    
        Labels = np.asarray(Labels, dtype=np.int32)
        model = cv2.face.LBPHFaceRecognizer_create()
        model.train(np.asarray(Training_Data), Labels)
        model.write('./model.yml')



    def open_login_window(self):
        self.login_window = tk.Toplevel(self.root)
        self.login_window.title("Вход")

        tk.Label(self.login_window, text="Логин:").pack()
        self.login_entry = tk.Entry(self.login_window)
        self.login_entry.pack()

        tk.Label(self.login_window, text="Пароль:").pack()
        self.password_entry = tk.Entry(self.login_window, show='*')
        self.password_entry.pack()

        self.show_password = tk.BooleanVar()
        self.show_password_checkbox = tk.Checkbutton(self.login_window, text="Показать пароль", variable=self.show_password, command=self.toggle_password)
        self.show_password_checkbox.pack()

        self.login_confirm_button = tk.Button(self.login_window, text="Подтвердить", command=self.login_user)
        self.login_confirm_button.pack(pady=10)

    def toggle_password(self):
        if self.show_password.get():
            self.password_entry.config(show='')
        else:
            self.password_entry.config(show='*')

    def login_user(self):
        login = self.login_entry.get().strip()  # Удаляем пробелы в начале и конце
        password = self.password_entry.get().strip()  # Удаляем пробелы в начале и конце
    
        if not login or not password:
            messagebox.showwarning("Warning", "Пожалуйста, заполните все поля!")
            return
    
        users = pd.read_csv(self.data_file)
        user = users[users['login'] == login]
    
        if user.empty:
            messagebox.showerror("Error", "Неверный логин или пароль!")
            print(f"Login attempt with non-existing login: {login}")
            return
    
        # Проверка пароля с удалением пробелов
        stored_password = user.iloc[0]['password'].strip()  # Удаляем пробелы
        if stored_password != password:
            messagebox.showerror("Error", "Неверный логин или пароль!")
            print(f"Password mismatch for login: {login}. Entered: {password}, Stored: {stored_password}")
            return
    
        print(f"Login successful for: {login}")  # Отладочное сообщение
        user_id = user.iloc[0]['ID']
        self.verify_face(user_id)



    def verify_face(self, user_id):
        print(f"Verifying face for user ID: {user_id}")  # Отладочное сообщение
        cap = cv2.VideoCapture(0)
        start_time = cv2.getTickCount()
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read('./model.yml')
    
        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Не удалось захватить кадр.")
                break
    
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_classifier.detectMultiScale(gray, 1.3, 5)
    
            print(f"Detected faces: {len(faces)}")  # Отладочное сообщение
    
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
                face = gray[y:y+h, x:x+w]
                face = cv2.resize(face, (200, 200))
    
                try:
                    results = model.predict(face)
    
                    # Убедимся, что ID совпадает с ID пользователя
                    predicted_id = results[0]
                    confidence = int(100 * (1 - (results[1] / 310)))
    
                    # Проверяем, что уверенность достаточна и ID совпадает
                    if confidence > 90 and str(predicted_id) == str(user_id):  # 90% confidence threshold
                        messagebox.showinfo("Info", "Авторизация успешна!")
                        cap.release()
                        cv2.destroyAllWindows()
                        return
                    else:
                        messagebox.showerror("Error", "Лицо не соответствует учетной записи.")
                        break
    
                except Exception as err:
                    print(err)
    
            cv2.imshow("Face Verification", frame)
    
            elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
            if elapsed_time > 30:  # 30 seconds timeout
                messagebox.showwarning("Warning", "Время вышло, попробуйте снова.")
                break
    
            if cv2.waitKey(1) == 27:  # ESC для выхода
                break
    
        cap.release()
        cv2.destroyAllWindows()




if __name__ == "__main__":
    root = tk.Tk()
    app = MultiFactorAuthApp(root)
    root.mainloop()